## NER Notebook

In [2]:
# download commands if needed
#!pip install jaro-winkler
#!pip install spacy
#!python -m spacy download en_core_web_lg

# load base libraries
import pandas as pd
import numpy as np
import re
import csv
import pickle
import jaro
from itertools import combinations

# load spacy libraries 
import spacy 
from spacy import displacy
import en_core_web_lg
nlp = spacy.load("en_core_web_lg")

### Tasks:
1. For each news feed, identify Organization, People and location
2. For recognized Organizations, identify repeating entities using a string distance measure
3. For each unique orgnization, obtain frequency, surface the top 2 recognized organizations

In [3]:
# alt startup for M&A DUNS only
devdata_multilabel = pd.read_excel('devdata_narrower_businessevents.xlsx') 
devdata_multilabel = devdata_multilabel[devdata_multilabel['label'] == 'acquisitions-mergers-and-takeovers']
devdata_multilabel = devdata_multilabel.reset_index(drop=True)

In [208]:
# load data
devdata_multilabel = pd.read_excel('devdata_narrower_businessevents.xlsx') 
devdata_multilabel = devdata_multilabel[devdata_multilabel['label'] != 'obituaries'] # remove "obituaries" category
devdata_multilabel = devdata_multilabel.reset_index(drop=True)

In [4]:
# process data
# only include body text -> headlines confuse NER as it is case sesitive
# remove common punctuation (comma, period) and possesives ('s)
#devdata_multilabel['text'] = devdata_multilabel['headline'] + '. ' + devdata_multilabel['body'] # combine headline and body of text
devdata_multilabel['text'] = devdata_multilabel['body'] # only use body
devdata_multilabel['text'] = devdata_multilabel['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True) # remove line breaker
devdata_multilabel['text'] = devdata_multilabel['text'].replace(',','', regex=True).replace('\.','', regex=True)  # remove punctuation
devdata_multilabel['text'] = devdata_multilabel['text'].replace('\'s','', regex=True).replace('\'','', regex=True) # remove posesives 
devdata_multilabel['text'] = devdata_multilabel['text'].replace('\'','', regex=True).replace(':',' ', regex=True)
devdata_multilabel = devdata_multilabel[['label','text']] # keep only two columns

### Name Entity Recognition Function

In [5]:
# define ner fuction for single news article
def ner(newsfeed):
    
    # run spacy nlp fuction
    news_nlp = nlp(newsfeed)
    
    # extract identified organizational entities
    recognized_org = []
    for entity in news_nlp.ents:
        label = entity.label_
        if label == 'ORG': recognized_org.append(entity.text)
        else: continue
    
    # fuzzy-string comparission
    pairs = list(combinations(recognized_org,2))
    repeating = []
    for i in pairs:
         if jaro.jaro_winkler_metric(i[0].lower(), i[1].lower()) > 0.8:
            if len(i[0]) < len(i[1]): i = [i[1], i[0]] # want longest version in position 0
            if i[1] != i[0]: repeating.append(i) # only keep non-same pairs

    # fix repeating entities
    for i in repeating: newsfeed = newsfeed.replace(i[0],i[1])
    
    # save identified entities
    news_nlp = nlp(newsfeed)
    recognized_org = []
    recognized_gpe = []
    recognized_date = []
    recognized_people = []

    for entity in news_nlp.ents:
        label = entity.label_
        if label == 'ORG': recognized_org.append(entity.text)
        elif label == 'DATE': recognized_date.append(entity.text)
        elif label == 'GPE': recognized_gpe.append(entity.text)
        elif label == 'PERSON': recognized_people.append(entity.text)
        else: continue
    
    # output identified entities
    newsfeed_entities = pd.DataFrame([[recognized_org], [recognized_gpe], [recognized_date], [recognized_people]])
    
    # handel no organizational entities
    if len(recognized_org) > 0:
        # count unique organizations frequency
        wordfreq = []
        for org in recognized_org: wordfreq.append(newsfeed.count(org))
        
        # find top 2 most frequent organizations
        orgfreq = pd.DataFrame(zip(recognized_org, wordfreq))
        orgfreq = orgfreq.rename(columns={0:'entity',1:'count'})
        orgfreq = orgfreq.sort_values(by='count', ascending=False)
        orgfreq = orgfreq.drop_duplicates().reset_index(drop=True)

        # output top 2 most frequent organizations
        try: top2 = [(orgfreq.iloc[0,0], orgfreq.iloc[0,1]), (orgfreq.iloc[1,0], orgfreq.iloc[1,1])]
        except: top2 = [(orgfreq.iloc[0,0], orgfreq.iloc[0,1])] # handel if only one organization
        newsfeed_entities = newsfeed_entities.append([[top2]])
    else:
        newsfeed_entities = newsfeed_entities.append([])
    
    # output final result
    newsfeed_entities = newsfeed_entities.reset_index(drop=True)
    return newsfeed_entities, repeating

newsfeed = devdata_multilabel['text'].iloc[1]
#newsfeed_entities, repeating = ner(newsfeed)

In [6]:
# run ner function on all news articles (17432 articles total)
entities = pd.DataFrame()
repeats = pd.DataFrame()
i = 0
for feed in devdata_multilabel['text'][:200]:
    newsfeed_entities, repeating = ner(feed)
    entities = entities.append(newsfeed_entities.T)
    repeats = repeats.append(repeating)
    print(i)
    i += 1
    
# clean output datafame
entities = entities.rename(columns={0:'Organization',1:'Geolocation',2:'Datetime',3:'Person',4:'Top2'})
entities = entities.reset_index(drop=True)
repeats = repeats.reset_index(drop=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [7]:
entities

,Organization,Geolocation,Datetime,Person,Top2
0,"[RALEIGH NC, Rifiniti]",[],"[July 10 2019, today, today]",[],"[(RALEIGH NC, 1), (Rifiniti, 1)]"
1,"[Premium Transportation Logistics, Senior Mana...",[TOLEDO],"[July 1 2019, today]","[Jeff Curry, Keith Avery, Brad Kelley]","[(PTL, 4), (Premium Transportation Logistics, 1)]"
2,"[ANNAPOLIS Md, Chesapeake Eye Care, Vision Inn...",[],[June 24 2019],[Michael Dunn],"[(Vision Innovation Partners, 3), (ANNAPOLIS M..."
3,"[GREENWICH Conn, Southfield Capital, Ntiva Inc...","[Chicago, Illinois]","[July 30 2019, today]",[],"[(GREENWICH Conn, 1), (Southfield Capital, 1)]"
4,"[Great Wave, Salesforce, Ksquare, Glenn Kohner...",[DALLAS],[20 2019],[Ksquare],"[(Ksquare, 3), (Great Wave, 2)]"
...,...,...,...,...,...
195,"[Jazz Pharmaceuticals plc, Nasdaq JAZZ, Redx ...",[DUBLIN],"[July 10 2019, today]",[],"[(Redx, 3), (RAF, 2)]"
196,"[GREENWICH Conn, Blue Harbour Group LP, BCA, B...",[],"[June 26 2019, 2018, today]",[],"[(BCA, 2), (GREENWICH Conn, 1)]"
197,"[BEDFORD Mass, Great Hill Partners]",[],"[June 26 2019, today]",[],"[(BEDFORD Mass, 1), (Great Hill Partners, 1)]"
198,"[Archcon Architecture, Archcon Design Build, D...","[SAN ANTONIO, Texas]",[July 25 2019],[Cheryl Cole],"[(DALLENBACH, 2), (Archcon Architecture, 1)]"


In [8]:
# get all unique orgs
allOrgs = pd.DataFrame()
for orgs in entities['Organization'][:]:
    allOrgs = allOrgs.append(orgs)

# process all orgs for duplicates
allOrgs = allOrgs.drop_duplicates().reset_index(drop=True)

In [9]:
# handle repeats and alternative namings
for i in range(allOrgs.shape[0]): # for each unique organization
    org = allOrgs.iloc[i,0]
    for j in range(repeats.shape[0]): # for each similar organization
        repeat = repeats.iloc[j,0]
        if org == repeat: # are they the same?
            allOrgs.iloc[i,0] = repeats.iloc[j,1] # repalce with shortened, equivilant org
            #print(i, org, j, repeats.iloc[j,1])

In [10]:
# huge loop - only run if really needed - catches similar orgs across articles
# fuzzy-string comparission for allOrgs
check_repeats = False
if check_repeats:
    allOrgs = allOrgs.drop_duplicates().reset_index(drop=True)
    pairs = list(combinations(list(allOrgs[0]),2))
    repeating = []
    for i in pairs:
         if jaro.jaro_winkler_metric(i[0].lower(), i[1].lower()) > 0.8:
                if len(i[0]) < len(i[1]): i = [i[1], i[0]] # want longest version in position 0
                repeating.append(i)
    repeating = pd.DataFrame(repeating)

In [11]:
# define aggregation function
def agg_fun(x):
    agg = []
    for i in x: agg += [i]
    return agg

# aggregate similar organization entities
if check_repeats: repeat_df = repeats.append(repeating).reset_index(drop=True)
else: repeat_df = repeats.copy()
repeat_dic = repeat_df.groupby(1).agg(agg_fun).reset_index() 

# process all orgs for new duplicates and merge with alt name df
allOrgs = allOrgs.drop_duplicates().reset_index(drop=True)
unqOrg = allOrgs.merge(repeat_dic, left_on=0, right_on=1, how='outer')
unqOrg = unqOrg.drop(columns=[1,'0_x'])
unqOrg = unqOrg.reset_index().rename(columns={'index':'DUNS','key_0':'short_entity_name','0_y':'alt_name_list'}) 
unqOrg['alt_name_list'] = unqOrg['alt_name_list'] + unqOrg['short_entity_name'].apply(lambda x: [x])   
unqOrg

,DUNS,short_entity_name,alt_name_list
0,0,RALEIGH NC,NaN
1,1,Rifiniti,NaN
2,2,Premium Transportation Logistics,NaN
3,3,Senior Management Team,NaN
4,4,PTL,NaN
...,...,...,...
607,607,Sierra MCC,"[Sierra Income Corporation, Sierra MCC]"
608,608,Spark Power Corp,"[Spark Power Group Inc, Spark Power Corp]"
609,609,VEON Holdings,"[VEON Holdings BV, VEON Holdings]"
610,610,Wacoal America Inc,"[Wacoal International Corporation, Wacoal Amer..."


In [12]:
# run to save as DUNS number file
unqOrg.to_csv('mna_duns.csv', index=False)